In [ ]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


# Load JSON data from a file
with open('anthony_intents.json', 'r') as file:
    data = json.load(file)

# Initialize lists to store different parts of the data
tags = []
patterns = []
responses = []
intents_list = []

for intent in data['intents']:
   for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
        responses.append((intent['tag'], intent['responses']))
intents_list.append(intent)

# Example: Printing the lists
print(tags[:2])  # Print first two tags to demonstrate
print(patterns[:4])  # Print first four patterns to demonstrate
print(responses[:4])  # Print first four responses to demonstrate

['greeting', 'greeting']
['Hi', 'Hey', 'Is anyone there?', 'Hello']
[('greeting', ['Hello', 'Hi', 'Hi there', 'Hello how are you?']), ('greeting', ['Hello', 'Hi', 'Hi there', 'Hello how are you?']), ('greeting', ['Hello', 'Hi', 'Hi there', 'Hello how are you?']), ('greeting', ['Hello', 'Hi', 'Hi there', 'Hello how are you?'])]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encoding the intents
label_encoder = LabelEncoder()
intent_labels = label_encoder.fit_transform(tags)

print("Encoded Intents:", intent_labels)
print("Mapping:", {label: idx for idx, label in enumerate(label_encoder.classes_)})


Encoded Intents: [ 6  6  6  6  6 17 17 17 17 14 14 14 14 14  7  7  7  7 10 10 10 10  1  1
  1  1  2  2  2  2 12 12 12 12 18 18 18 18 13 13 13 13  0  0  0  0  9  9
  9  9  5  5  5  5 15 15 15 15  4  4  4  4 16 16 16 16  8  8  8  8  3  3
  3  3 11 11 11 11]
Mapping: {'allergy_info': 0, 'beverage_recommendation': 1, 'dietary_restrictions': 2, 'drink_preparation_time': 3, 'environmental_efforts': 4, 'gift_cards': 5, 'greeting': 6, 'handling_complaints': 7, 'job_opportunities': 8, 'loyalty_program': 9, 'menu_inquiry': 10, 'mobile_ordering': 11, 'operational_hours': 12, 'order_customization': 13, 'ordering_coffee': 14, 'private_events': 15, 'product_availability': 16, 'thanks': 17, 'wifi_access': 18}


In [ ]:
# Tokenizing the patterns
tokenizer = Tokenizer(num_words=900, oov_token="<OOV>")
tokenizer.fit_on_texts(patterns)

# Creating sequences
sequences = tokenizer.texts_to_sequences(patterns)

# Padding sequences
padded_sequences = pad_sequences(sequences, maxlen=40, padding='post', truncating='post')

print("Word Index:", tokenizer.word_index)
print("Sequences:", sequences[:5])  # Showing first 5 sequences as example
print("Padded Sequences:", padded_sequences[:5])  # Showing padded version of first 5 sequences

Word Index: {'<OOV>': 1, 'you': 2, 'i': 3, 'do': 4, 'the': 5, 'can': 6, 'a': 7, 'are': 8, 'my': 9, 'is': 10, 'how': 11, 'for': 12, 'your': 13, 'coffee': 14, 'with': 15, 'what': 16, 'there': 17, 'have': 18, 'order': 19, 'get': 20, 'drink': 21, 'any': 22, 'latte': 23, 'an': 24, "what's": 25, 'free': 26, 'loyalty': 27, 'gift': 28, 'to': 29, 'milk': 30, 'espresso': 31, 'on': 32, 'pastries': 33, 'offer': 34, 'open': 35, 'time': 36, 'wifi': 37, 'use': 38, 'available': 39, 'thanks': 40, "i'd": 41, 'like': 42, 'cappuccino': 43, 'please': 44, 'menu': 45, 'me': 46, 'options': 47, 'recommend': 48, 'drinks': 49, 'vegan': 50, 'gluten': 51, 'dairy': 52, 'sugar': 53, 'when': 54, 'in': 55, 'make': 56, 'it': 57, 'program': 58, 'points': 59, 'check': 60, 'cards': 61, 'card': 62, 'online': 63, 'event': 64, 'events': 65, 'positions': 66, 'long': 67, 'does': 68, 'wait': 69, 'app': 70, 'ordering': 71, 'hi': 72, 'hey': 73, 'anyone': 74, 'hello': 75, 'thank': 76, "that's": 77, 'helpful': 78, 'help': 79, 'want

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

num_classes = 19

model = Sequential([
    Embedding(input_dim=900, output_dim=20, input_length=40),  # Embedding layer
    GlobalAveragePooling1D(),  # Global average pooling layer
    Dense(16, activation='relu'),  # First dense layer
    Dense(10, activation='sigmoid'),  # Second dense layer
    Dense(num_classes, activation='softmax')  # Final dense layer for classification
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 20)            18000     
                                                                 
 global_average_pooling1d_5  (None, 20)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_15 (Dense)            (None, 16)                336       
                                                                 
 dense_16 (Dense)            (None, 10)                170       
                                                                 
 dense_17 (Dense)            (None, 19)                209       
                                                                 
Total params: 18715 (73.11 KB)
Trainable params: 18715 (73.11 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [ ]:
X_train = padded_sequences
y_train = intent_labels

# Train with 500 epochs
history_500 = model.fit(X_train, y_train, epochs=500, verbose=1)

Epoch 1/500
3/3 [==============================] - 1s 7ms/step - loss: 2.9976 - accuracy: 0.0641
Epoch 2/500
3/3 [==============================] - 0s 5ms/step - loss: 2.9930 - accuracy: 0.0641
Epoch 3/500
3/3 [==============================] - 0s 6ms/step - loss: 2.9914 - accuracy: 0.0641
Epoch 4/500
3/3 [==============================] - 0s 6ms/step - loss: 2.9891 - accuracy: 0.0641
Epoch 5/500
3/3 [==============================] - 0s 6ms/step - loss: 2.9870 - accuracy: 0.0641
Epoch 6/500
3/3 [==============================] - 0s 6ms/step - loss: 2.9851 - accuracy: 0.0641
Epoch 7/500
3/3 [==============================] - 0s 6ms/step - loss: 2.9832 - accuracy: 0.0641
Epoch 8/500
3/3 [==============================] - 0s 6ms/step - loss: 2.9815 - accuracy: 0.0641
Epoch 9/500
3/3 [==============================] - 0s 6ms/step - loss: 2.9795 - accuracy: 0.0641
Epoch 10/500
3/3 [==============================] - 0s 5ms/step - loss: 2.9780 - accuracy: 0.0641
Epoch 11/500
3/3 [===========

In [ ]:
# Train with 1000 epochs
history_1000 = model.fit(X_train, y_train, epochs=1000, verbose=1)

Epoch 1/1000
3/3 [==============================] - 0s 7ms/step - loss: 1.3799 - accuracy: 0.8590
Epoch 2/1000
3/3 [==============================] - 0s 6ms/step - loss: 1.3774 - accuracy: 0.8462
Epoch 3/1000
3/3 [==============================] - 0s 7ms/step - loss: 1.3752 - accuracy: 0.8333
Epoch 4/1000
3/3 [==============================] - 0s 8ms/step - loss: 1.3729 - accuracy: 0.8077
Epoch 5/1000
3/3 [==============================] - 0s 6ms/step - loss: 1.3704 - accuracy: 0.8077
Epoch 6/1000
3/3 [==============================] - 0s 6ms/step - loss: 1.3682 - accuracy: 0.8205
Epoch 7/1000
3/3 [==============================] - 0s 8ms/step - loss: 1.3658 - accuracy: 0.8077
Epoch 8/1000
3/3 [==============================] - 0s 6ms/step - loss: 1.3633 - accuracy: 0.8077
Epoch 9/1000
3/3 [==============================] - 0s 7ms/step - loss: 1.3611 - accuracy: 0.8077
Epoch 10/1000
3/3 [==============================] - 0s 6ms/step - loss: 1.3587 - accuracy: 0.8205
Epoch 11/1000
3/3 [

In [ ]:
accuracy_500 = history_500.history['accuracy'][-1]
accuracy_1000 = history_1000.history['accuracy'][-1]

print(f"Accuracy after 500 epochs: {accuracy_500}")
print(f"Accuracy after 1000 epochs: {accuracy_1000}")

Accuracy after 500 epochs: 0.8589743375778198
Accuracy after 1000 epochs: 1.0


In [ ]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
model.save('Anthony_model')

TESTING THE BOT

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import json
import random

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the label encoder
with open('label_encoder.pickle', 'rb') as enc_file:
    label_encoder = pickle.load(enc_file)

# Load the model
model = tf.keras.models.load_model('Anthony_model')

# Load the intents JSON file
with open('anthony_intents.json') as file:
    intents = json.load(file)

def get_response(intent_tag, intents_json):
    for intent in intents_json["intents"]:
        if intent["tag"] == intent_tag:
            return random.choice(intent["responses"])
    return "I'm not sure how to answer that."

def chat():
    print("Start talking with the bot (type 'bye' to stop)!")
    while True:
        # Receive input from the user
        inp = input("You: ")
        if inp.lower() == "bye":
            break

        # Pre-process the input
        seq = tokenizer.texts_to_sequences([inp])
        padded = pad_sequences(seq, maxlen=40, padding='post')

        # Predict the intent
        predictions = model.predict(padded)
        intent_index = predictions.argmax()
        intent_tag = label_encoder.inverse_transform([intent_index])[0]

        # Get and print the response
        response = get_response(intent_tag, intents)
        print("Bot:", response)




In [ ]:
# Run the chat
chat()

Start talking with the bot (type 'bye' to stop)!
You: hi
1/1 [==============================] - 0s 22ms/step
Bot: Hi
You: I'd like to order a coffee
1/1 [==============================] - 0s 27ms/step
Bot: Absolutely, I'll have that ready for you in a moment. Anything else to add?
You: Do you have any specials?
1/1 [==============================] - 0s 21ms/step
Bot: Today's special is a Pumpkin Spice Latte.
You: Recommend me a drink please.
1/1 [==============================] - 0s 21ms/step
Bot: Our Cold Brew is a hit during summer days!
You: Do you have any vegan options?
1/1 [==============================] - 0s 20ms/step
Bot: Yes, we have almond and soy milk as dairy-free alternatives.
You: I want a cappuccino with almond milk
1/1 [==============================] - 0s 22ms/step
Bot: Absolutely, I'll have that ready for you in a moment. Anything else to add?
You: Can I get extra shot in my latte?
1/1 [==============================] - 0s 34ms/step
Bot: Yes, we can prepare it with d